In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import time
import urllib
import re
import ast

In [2]:
# Reclaim login credentials for chicago cityscape
email = "reclaimfairelections@gmail.com"
password = "rDxPK7p6W@5f"
main_page = 'https://www.chicagocityscape.com'
ord_page = 'https://www.chicagocityscape.com/ordinances.php'
ord_num = 'O2019-331'
ord_dict = {}

In [3]:
def initialize_driver(main_page):
    driver = webdriver.Firefox()
    driver.get(main_page)
    wait = WebDriverWait(driver, 30)
    return driver, wait

def login_cityscape(driver, wait, email, password):  
    # Find sign-in by Google button and click it
    elem = driver.find_element_by_partial_link_text('Sign in')
    elem.click()
    time.sleep(2)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "fa-ul")))
    # Find google sign in
    goo = driver.find_element_by_partial_link_text('Google')
    goo.click()
    time.sleep(2)

    # Enter Email address and submit
    wait.until(EC.presence_of_element_located((By.ID, "headingText")))
    email_input = driver.find_element_by_id("identifierId")
    email_input.send_keys(email)
    email_submit = driver.find_element_by_id("identifierNext").click()
    time.sleep(2)

    
    # Enter Password and submit
    wait.until(EC.presence_of_element_located((By.ID, "password")))
    pw_input = driver.find_element_by_name("password")
    pw_input.send_keys(password)
    pw_submit = driver.find_element_by_id("passwordNext").click()
    time.sleep(10)
    
    return driver, wait

In [4]:
def search_ord(driver, wait, ord_page, ord_num):
    # Go to ordinance search page
    driver.get(ord_page)
    
    # Input ord num in search bar
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "form-inline")))
    ord_input = driver.find_element_by_id('search_term')
    ord_input.send_keys(ord_num)
    key = driver.find_element_by_class_name('btn')
    key.click()
    time.sleep(2)

    # Find Details link for ordinance
    wait.until(EC.presence_of_element_located((By.ID, "table_ordinances_wrapper")))
    ord_link = driver.find_element_by_partial_link_text('Details')
    ord_link.click()
    time.sleep(10)
    return driver, wait

def find_ord_id(driver, ord_page):
    # get search page html to find ordinance id
    ord_details = driver.page_source
    soup = BeautifulSoup(ord_details, "html5lib")
    link = ''
    for s in soup.find_all('a'):
        links = s.get('href')
        if links:
            if '/ordinances.php?ordinance=' in links:
                link = links
    ord_code = link.split('php')[1]
    ord_code_page = ord_page + ord_code
    return driver, ord_code_page

In [5]:
def get_ord_html(driver, ord_page):
    driver.get(ord_code_page)
    html = driver.page_source
    soup = BeautifulSoup(html, "html5lib")
    return soup

def get_ord_summary(soup, ord_dict):
    # get ordinance summary text; add it to dict
    for s in soup.find_all('h3'):
        if s.next_element == 'Ordinance summary ':
            ord_dict['summary'] = s.next_sibling.next_sibling.text
    return ord_dict 

def get_ord_players(soup, ord_dict):
    # extract 'more details and place in dict', including key players
    for s in soup.find_all('dl', class_='dl-horizontal'):
        for d in s.find_all('dt'):
            ord_dict[d.text] = d.next_sibling.text  
    return ord_dict     

def get_ord_places(soup, ord_dict):
    place_list = ['Community Area', 'Neighborhood', 'ZIP Code']
    ward_list = ['Ward (Chicago)']
    for s in soup.find_all('div', id='boundaries_printer_wrapper'):
        for t in s.find_all('td', class_='sorting_3'):
            place = re.split('\s\s',t.text)[0]
            category = re.split('\s\s',t.next_sibling.text)[0]
            place = place.split(',')
            if category in place_list:
                ord_dict[category] = place
            if category in ward_list:
                ord_dict['Ward'] = place[0]
                ord_dict['Ward_Alderman'] = place[1]
    return ord_dict

def get_all_ord_info(driver, ord_code_page, ord_dict):
    soup = get_ord_html(driver, ord_page)
    ord_dict = get_ord_summary(soup, ord_dict)
    ord_dict = get_ord_players(soup, ord_dict)
    ord_dict = get_ord_places(soup, ord_dict)
    return ord_dict

In [6]:
# Run code here
driver, wait = initialize_driver(main_page)
driver, wait = login_cityscape(driver, wait, email, password)


driver, wait = search_ord(driver, wait, ord_page, ord_num)
driver, ord_code_page = find_ord_id(driver, ord_page)
ord_dict = get_all_ord_info(driver, ord_code_page, ord_dict)

In [9]:
ord_dict

{'summary': 'Construction of a residential development containing 152 dwelling units and 179 parking spaces, together with accessory and incidental uses.',
 'Architect': '',
 'Owner': 'Seritage SRC Finance LLC',
 'Applicant': 'Seritage SRC Finance LLC',
 'Lawyer': 'Rich Klawiter & Katie Jahnke Dale',
 'Law firm': 'DLA Piper LLP (US)',
 'Community Area': ['Austin'],
 'Neighborhood': ['Galewood'],
 'Ward': '29th Ward',
 'Ward_Alderman': ' Alder Chris Taliaferro',
 'ZIP Code': ['60707']}

In [31]:
# open data from chicouncilmatic
with open('ord_dicts.txt') as f:
    data = ast.literal_eval(f.read())

In [43]:
# open partial data file from cityscape
ord_nums_complete = []
with open('ord_info_final.txt') as f:
    ord_complete = ast.literal_eval(f.read())

240

In [46]:
# append each item to ord_list
ord_list = []
for item in ord_complete:
    ord_list.append(item)
    ord_nums_complete.append(item['ord'])    

In [ ]:
# Initialize driver & log in to cityscape
driver, wait = initialize_driver(main_page)
driver, wait = login_cityscape(driver, wait, email, password)

# Iterate through ord numbers
for entry in data:
    ord_num = entry['ord']
    
    # skip ords that are already in text file
    if ord_num in ord_nums_complete:
        continue;
    
    try:
        driver, wait = search_ord(driver, wait, ord_page, ord_num)
        driver, ord_code_page = find_ord_id(driver, ord_page)
        entry = get_all_ord_info(driver, ord_code_page, entry)
    
    except: 
        continue
    ord_list.append(entry)


O2019-278
O2019-282
o2018-9268
O2018-6941
O2019-271
O2018-9319
O2018-9308
O2018-9297
O2019-261
O2018-8057
O2019-269
O2018-8387
O2017-1924
O2016-5571
O2019-308
O2018-8049
O2018-9032
O2019-276
O2019-298
O2018-8393
O2019-281
O2019-274
O2019-333
O2018-6011
O2019-322
O2019-277
O2019-270
O2019-323
O2019-1153
O2019-265
O2019-332
O2018-9277
O2019-275
O2019-324
O2019-307
O2019-321
O2018-9282
O2018-9269
O2018-9280
O2019-365
O2019-267
O2018-8010
O2018-9265
O2019-283
O2018-9311
O2019-294
O2019-374
O2019-263
O2018-8021
O2019-329
O2018-6013
O2018-8169
O2018-9459
O2019-262
O2019-328
O2019-260
O2018-9343
O2018-9278
O2018-9279
O2019-309
O2019-264
O2018-8079
O2019-331
O2018-7750
O2019-266
O2019-325
O2018-9016
O2018-8410
O2018-9124
O2018-8020
O2018-9492
O2019-279
O2018-8006
O2018-8389
O2018-9266
O2018-9267
O2019-278
O2019-282
O2018-9268
O2019-269
O2018-9263
O2018-8017
O2018-9254
O2018-9447
O2018-9486
O2018-8077
O2018-9453
O2018-8394
O2018-9475
O2018-9890
O2018-4455
O2018-5980
O2018-9260
O2018-9551
O2018-

In [40]:
import json
with open('ord_info_final.txt', 'w') as fout:
    fout.write(json.dumps(ord_list, indent=4, sort_keys=True, default=str))